In [6]:
# Import libs
import numpy as np
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

import requests 


### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'CRYPL3E3LQZ2W2TQO5MXB4VKFKMMNKRW1BORN3UNLCMMOHOT'
CLIENT_SECRET = 'VM41FNBBEBYZUZKAK413IMIYRXQZKWNOIDFTO5EMUMNE4NXR'
VERSION = '20180604'
LIMIT = 10
search_query = "cafe"
radius = 2000

address = 'Toronto, Ontario, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

url = 'https://api.foursquare.com/v2/venues/suggestcompletion?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude,
            search_query,
            radius, 
            LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f05dba328796f3c9c0637ba'},
 'response': {'minivenues': [{'id': '4b06ef4cf964a52072f322e3',
    'name': 'Cafe On The Square',
    'location': {'address': '100 Queen St W',
     'city': 'Toronto',
     'state': 'ON',
     'postalCode': 'M5H 2N2',
     'country': 'CA',
     'lat': 43.65263300870971,
     'lng': -79.38336074020555,
     'distance': 105},
    'categories': [{'id': '4bf58dd8d48988d1c4941735',
      'name': 'Restaurant',
      'pluralName': 'Restaurants',
      'shortName': 'Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True},
     {'id': '4d4b7105d754a06374d81259',
      'name': 'Food',
      'pluralName': 'Food',
      'shortName': 'Food',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'}}]},
   {'id': '4b7adeeaf964a5208a422fe3',
    'name': 'Trinity Square Cafe',
    'location': {'city': 'To